In [31]:
import pandas as pd
import numpy as np

clean_data = pd.read_csv('./1211_total.csv')
print(clean_data.head())

dataDir='./'
output_file= dataDir + 'test2_new.txt'
fOut = open(output_file, 'r')
truth_df = []
for line in fOut:
    arr_test=line.strip().split('|')
    truth_df_rows = [arr_test[0], arr_test[1], arr_test[2]]
    truth_df.append(truth_df_rows)
fOut.close()
    
col_header = ['UserID', 'TrackID', 'Truth']
truth_df = np.array(truth_df)
truth_df = pd.DataFrame(truth_df)
truth_df.columns = col_header
print(truth_df.head())
print(len(truth_df))

ratings_supervised = []
ratings_unsupervised = []
saved_rows = []
truth_arr = []
    
userID_truth_arr = truth_df['UserID'].values
userID_truth_arr = userID_truth_arr.astype(np.float64)
data = clean_data

for row in range(1, len(data)):
    data_userID = data['UserID'][row]
    data_userID = float(data_userID)
    if (data_userID in userID_truth_arr):
        ratings_super = [data['TrackID'][row], data['UserID'][row], data['ItemID'][row], data['AlbumID'][row], data['ArtistID'][row], data['Genre1ID'][row], data['Genre2ID'][row], data['Genre3ID'][row], data['AlbumRating'][row], data['ArtistRating'][row], data['Genre1Rating'][row], data['Genre2Rating'][row], data['Genre3Rating'][row], data['MinScore'][row], data['MaxScore'][row], data['MeanScore'][row], data['VarScore'][row]]
        ratings_supervised.append(ratings_super) 

ratings_supervised = np.array(ratings_supervised)
super_col = ['TrackID', 'UserID', 'ItemID', 'AlbumID', 'ArtistID', 'Genre1ID', 'Genre2ID', 'Genre3ID', 'AlbumRatin', 'ArtistRating', 'Genre1Rating', 'Genre2Rating', 'Genre3Rating', 'MinScore', 'MaxScore', 'MeanScore', 'VarScore']
ratings_supervised = pd.DataFrame(ratings_supervised, columns=super_col)

itemID_data_arr = ratings_supervised['ItemID'].values
itemID_data_arr = itemID_data_arr.astype(np.float64)

for i in range(0, len(truth_df)):
    data_itemID = truth_df['TrackID'][i]
    data_itemID = float(data_itemID)
    truth = truth_df['Truth'][i]
    truth = float(truth)
    
    if (data_itemID in itemID_data_arr):
        truth_arr.append(truth)

ratings_supervised['Truth'] = truth_arr
print(ratings_supervised.head())
ratings_supervised.to_csv('./ratings_supervised_1211.csv', index=False)


   AlbumRating_MF  ArtistRating_MF  Genre1Rating_MF  Genre2Rating_MF  \
0       11.690904        12.474678        13.945157        12.992632   
1       77.389346        80.694513        83.869803        79.879048   
2       14.254794        13.852148        10.914258        11.422376   
3        4.574390         4.590136         4.152230         4.137116   
4       72.024521        69.910350        54.788498        57.453849   

   Genre3Rating_MF        TrackID    UserID    ItemID   AlbumID  ArtistID  \
0        13.243756  199810_208019  199810.0  208019.0  209288.0       0.0   
1        80.586610   199810_74139  199810.0   74139.0  277282.0  271146.0   
2        11.039877    199810_9903  199810.0    9903.0       0.0       0.0   
3         4.087868  199810_242681  199810.0  242681.0  190640.0  244574.0   
4        55.480790   199810_18515  199810.0   18515.0  146344.0   33168.0   

   ...  Genre2Rating  Genre3Rating  MinScore  MaxScore  MeanScore  VarScore  \
0  ...           0.0     

In [34]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

test_ratings_supervised = ratings_supervised
test_final_project_test = clean_data
test_final_project_test = test_final_project_test.drop(['AlbumRating_MF', 'ArtistRating_MF', 'Genre1Rating_MF', 'Genre2Rating_MF', 'Genre3Rating_MF', 'MinScore_MF', 'MaxScore_MF', 'MeanScore_MF', 'VarScore_MF'], axis=1)
 
test_final_project_test.to_csv('./ratings_unsupervised_1211.csv', index=False)
train = test_ratings_supervised #train with whole dataset
test = test_ratings_supervised.loc[5700:]
print(train.shape)
print(test.shape)

y_train = train['Truth']
y_test = test['Truth']
X_train = train.drop(['Truth'], axis=1)
X_test = test.drop(['Truth'], axis=1)

clf1 = RandomForestClassifier(n_estimators=5000)
clf2 = GradientBoostingClassifier(n_estimators=5000)
clf3 = DecisionTreeClassifier()
clf4 = LogisticRegression(max_iter=5000)

#change classifier to the one you want
clf = clf1
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("acc training test: ", acc)

y_pred = clf.predict(test_final_project_test)
y_prob = clf.predict_proba(test_final_project_test)
y_prob = np.array(y_prob)
y_prob = np.transpose(y_prob)

new_data_frame = test_final_project_test[['TrackID']].copy()
new_data_frame['Predictor'] = y_pred
new_data_frame['prob_0'] = y_prob[0]
new_data_frame['prob_1'] = y_prob[1]
print(new_data_frame.head())

new_data_frame.to_csv('./rf_1211_cleaned_python_probabiliites.csv', index=False, header=['TrackID', 'Predictor', 'prob_0', 'prob_1'])


(6000, 18)
(300, 18)
acc training test:  1.0
         TrackID  Predictor    prob_0    prob_1
0  199810_208019        0.0  0.988229  0.011771
1   199810_74139        0.0  0.998597  0.001403
2    199810_9903        0.0  0.757274  0.242726
3  199810_242681        0.0  0.999979  0.000021
4   199810_18515        1.0  0.369203  0.630797


In [30]:
clean_new_data_frame = new_data_frame

new_predictors = []
new_predictors_index = 0
print(clean_new_data_frame.head())

for k,g in clean_new_data_frame.groupby(np.arange(len(clean_new_data_frame))//6):
    batched = g
    num_ones = 0
    num_zeros = 0
    index_count = 0
    index_one = []
    index_zero = []
    
    batch_predictor = batched['Predictor']
    batch_prob0 = batched['prob_0']
    batch_prob1 = batched['prob_1']
    
    for i in batch_predictor:
        new_predictors.append(i)
        new_predictors_index = new_predictors_index + 1
        if (i == 0):
            num_zeros = num_zeros + 1
            index_zero.append(index_count)
        if (i == 1):
            num_ones = num_ones + 1
            index_one.append(index_count)
        index_count = index_count + 1
    #print('num_zeros: ', num_zeros)
    
    index_count = 0
    index_rawPred_zero = []
    index_rawPred_one = []
    if (num_zeros > 3):
        for i in batch_prob0:
            if (index_count in index_zero):
                index_rawPred_zero.append(i)
            index_count = index_count + 1
            
        index_rawPred_zero.sort()
        index_rawPred_zero_toremove = []
        zeros_to_correct = num_zeros - 3
        for i in range(0, len(index_rawPred_zero)):
            if (zeros_to_correct > 0):
                index_rawPred_zero_toremove.append(index_rawPred_zero[i])
            zeros_to_correct = zeros_to_correct - 1
        index_count = 0
        index_to_switch = []
        for i in batch_prob0:
            if i in index_rawPred_zero_toremove:
                index_to_switch.append(index_count)
            index_count = index_count + 1
            
        index_count = 0
        for i in range(0, 6):
            if index_count in index_to_switch:
                new_predictors[k*6+index_count] = 1
            index_count = index_count + 1
            
    index_count = 0
    if (num_ones > 3):
        for i in batch_prob1:
            if (index_count in index_one):
                index_rawPred_one.append(i)
            index_count = index_count + 1
            
        index_rawPred_one.sort()
        index_rawPred_one_toremove = []
        ones_to_correct = num_ones - 3
        for i in range(0, len(index_rawPred_one)):
            if (ones_to_correct > 0):
                index_rawPred_one_toremove.append(index_rawPred_one[i])
            ones_to_correct = ones_to_correct - 1
        
        index_count = 0
        index_to_switch = []
        for i in batch_prob1:
            if i in index_rawPred_one_toremove:
                index_to_switch.append(index_count)
            index_count = index_count + 1
            
        index_count = 0
        for i in range(0, 6):
            if index_count in index_to_switch:
                new_predictors[k*6+index_count] = 0
            index_count = index_count + 1

clean_new_data_frame['Predictor'] = new_predictors
print(clean_new_data_frame.head())

unsuper_col = ['TrackID', 'Predictor']
clean_new_data_frame = clean_new_data_frame[['TrackID', 'Predictor']].copy()
print(clean_new_data_frame.head())
clean_new_data_frame.to_csv('./cleaned_rf_1211_cleaned.csv', index=False, header=unsuper_col)

         TrackID  Predictor    prob_0    prob_1
0  199810_208019        0.0  0.490470  0.509530
1   199810_74139        0.0  0.499047  0.500953
2    199810_9903        0.0  0.499905  0.500095
3  199810_242681        0.0  0.490470  0.509530
4   199810_18515        0.0  0.499047  0.500953
         TrackID  Predictor    prob_0    prob_1
0  199810_208019        1.0  0.490470  0.509530
1   199810_74139        0.0  0.499047  0.500953
2    199810_9903        0.0  0.499905  0.500095
3  199810_242681        1.0  0.490470  0.509530
4   199810_18515        0.0  0.499047  0.500953
         TrackID  Predictor
0  199810_208019        1.0
1   199810_74139        0.0
2    199810_9903        0.0
3  199810_242681        1.0
4   199810_18515        0.0
